In [ ]:
import nltk
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import metrics
%matplotlib inline
from google.colab import files
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score


In [ ]:
uploaded = files.upload()
data = pd.read_csv('dataset01.txt', sep = '\n')
print(data.head())

Saving dataset01.txt to dataset01.txt
                                            all_data
0                                     >iTELL-Chr1_61
1  CACACCACACCCACACACCCACACACCACACCACACACCACACCAC...
2                                                  0
3                                   >iYAL067W-A_5085
4  AATTTTTATAGGTCGACCCTTCTGTCGCTTACTGGGTTGATTATCT...


In [ ]:
all=data["all_data"].tolist()
cls=[]
sequence=[]
label=[]

In [ ]:
length = len(all)
for i in range(length):
   if (i%3)==0:
             cls.append(all[i])
   else:
      if(all[i]== "0" or all[i]=="1"):
        label.append(all[i])
      else:
         sequence.append(all[i])

print(len(cls))
print(len(label))
print(len(sequence))

14965
14965
14965


In [ ]:
df = pd.DataFrame(list(zip(cls, label, sequence)),
               columns =['class' , 'label', 'sequence'])
df.head(10)

,class,label,sequence
0,>iTELL-Chr1_61,0,CACACCACACCCACACACCCACACACCACACCACACACCACACCAC...
1,>iYAL067W-A_5085,0,AATTTTTATAGGTCGACCCTTCTGTCGCTTACTGGGTTGATTATCT...
2,>iYAL067W-A_6786,0,AATTATATTTCCATCAGCTCAATACCGCAGTACTTTGAAACCTGAT...
3,>iYAL067W-A_7060,0,AACAATAGTGGGTATGAGTAAAGATATATAGATCGATATTTTGAAT...
4,>YAL067C_SEO1_8061,1,CAAAGATTTCAACCATAGTAGGTATGCCCCAGATGAAACATTACTG...
5,>YAL067C_SEO1_8340,1,CACAATAGCGTCAATAATAAAGTTCCATCTCCATCCCTCTAAACCA...
6,>iYAL067C_9509,1,GTTTTTTTCCAGTGACACAATCTTTACCATTACACAGTTTTTACTA...
7,>iYAL067C_10442,0,CAGAAACATAAAATTATATCACTTTATTTCATATGGTTTCATGCTT...
8,>YAL065C_YAL065C_11570,1,AGATACCTCGAAACAAACTCTATGTAAACACTTATTTTATTGTGGT...
9,>iYAL064W-B_13286,1,ATGACTGGAGAAGAAGTGACTAGAGAAGAAGTGACTGAGGAAGAAA...


In [ ]:
X = df['sequence']
Y = df['label']

In [ ]:
def getKmers(sequence, size=6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]
    

In [ ]:
df['words'] = df.apply(lambda x: getKmers(x['sequence']), axis=1)
df = df.drop('sequence', axis=1)
df.head(10)             

,class,label,words
0,>iTELL-Chr1_61,0,"[cacacc, acacca, caccac, accaca, ccacac, cacac..."
1,>iYAL067W-A_5085,0,"[aatttt, attttt, ttttta, ttttat, tttata, ttata..."
2,>iYAL067W-A_6786,0,"[aattat, attata, ttatat, tatatt, atattt, tattt..."
3,>iYAL067W-A_7060,0,"[aacaat, acaata, caatag, aatagt, atagtg, tagtg..."
4,>YAL067C_SEO1_8061,1,"[caaaga, aaagat, aagatt, agattt, gatttc, atttc..."
5,>YAL067C_SEO1_8340,1,"[cacaat, acaata, caatag, aatagc, atagcg, tagcg..."
6,>iYAL067C_9509,1,"[gttttt, tttttt, tttttt, tttttc, ttttcc, tttcc..."
7,>iYAL067C_10442,0,"[cagaaa, agaaac, gaaaca, aaacat, aacata, acata..."
8,>YAL065C_YAL065C_11570,1,"[agatac, gatacc, atacct, tacctc, acctcg, cctcg..."
9,>iYAL064W-B_13286,1,"[atgact, tgactg, gactgg, actgga, ctggag, tggag..."


In [ ]:
dna_texts = list(df['words'])
for item in range(len(dna_texts)):
    dna_texts[item] = ' '.join(dna_texts[item])
y_data = df['label'].values

In [ ]:
cv = CountVectorizer(ngram_range=(4,4))
X = cv.fit_transform(dna_texts)
print(X.shape)

(14965, 261260)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_data, test_size = 0.25,  shuffle = True) 

print(X_train.shape)
print(X_test.shape)                                         

(11223, 261260)
(3742, 261260)


In [ ]:
scoring = 'accuracy'

names = ['K Nearest Neighbors', 'Decision Tree', 'Random Forest', 'AddaBoost', 'Multinomial NB',
         'SVM Linear', 'Logistic Regression']
Classifiers = [
    KNeighborsClassifier(n_neighbors = 3),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators = 100, max_features = 1 ),
    AdaBoostClassifier(),
    MultinomialNB(alpha=0.1),
    svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto'),
    LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=2, penalty='l2',max_iter=1000)
     ]

models = zip(names, Classifiers)
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(name)
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(accuracy_score(y_test, y_pred))


K Nearest Neighbors
[[1836   24]
 [1509  373]]
              precision    recall  f1-score   support

           0       0.55      0.99      0.71      1860
           1       0.94      0.20      0.33      1882

    accuracy                           0.59      3742
   macro avg       0.74      0.59      0.52      3742
weighted avg       0.75      0.59      0.52      3742

0.5903260288615714
Decision Tree
[[1406  454]
 [ 726 1156]]
              precision    recall  f1-score   support

           0       0.66      0.76      0.70      1860
           1       0.72      0.61      0.66      1882

    accuracy                           0.68      3742
   macro avg       0.69      0.69      0.68      3742
weighted avg       0.69      0.68      0.68      3742

0.6846606092998396
Random Forest
[[1471  389]
 [ 360 1522]]
              precision    recall  f1-score   support

           0       0.80      0.79      0.80      1860
           1       0.80      0.81      0.80      1882

    accuracy   

In [15]:
model_SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
model_SVM.fit(X_train, y_train) 
model_predictions = model_SVM.predict(X_test)
print("\n---------Support Vector Machine classifier---------")
print("\nConfusion matrix: \n", confusion_matrix(y_test,model_predictions))
print("\nClassification Report: \n", classification_report(y_test,model_predictions))
print("\nClassification accuracy: " ,accuracy_score(y_test, model_predictions) * 100, "%.\n")


---------Support Vector Machine classifier---------

Confusion matrix: 
 [[1590  270]
 [ 215 1667]]

Classification Report: 
               precision    recall  f1-score   support

           0       0.88      0.85      0.87      1860
           1       0.86      0.89      0.87      1882

    accuracy                           0.87      3742
   macro avg       0.87      0.87      0.87      3742
weighted avg       0.87      0.87      0.87      3742


Classification accuracy:  87.03901656867986 %.



In [16]:
from sklearn.linear_model import LogisticRegression
model_LR = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=2, penalty='l2',max_iter=1000)
model_LR.fit(X_train, y_train) 
model_predictions_LR = model_LR.predict(X_test)
print("\n---------Logistic regression classifier---------")
print("\nConfusion matrix: \n", confusion_matrix(y_test,model_predictions_LR))
print("\nClassification Report: \n", classification_report(y_test,model_predictions_LR))
print("\nClassification accuracy: " ,accuracy_score(y_test, model_predictions_LR) * 100, "%.\n")

[LibLinear]
---------Logistic regression classifier---------

Confusion matrix: 
 [[1605  255]
 [ 217 1665]]

Classification Report: 
               precision    recall  f1-score   support

           0       0.88      0.86      0.87      1860
           1       0.87      0.88      0.88      1882

    accuracy                           0.87      3742
   macro avg       0.87      0.87      0.87      3742
weighted avg       0.87      0.87      0.87      3742


Classification accuracy:  87.38642437199358 %.

